# Add Football Vars

In [1]:
import pandas as pd
import pickle

from datetime import datetime
from datetime import timedelta

In [2]:
# directories

# input
dataDir = '../../data/external/'
processDir = '../../data/processed/'
chargeDir = processDir + 'charges/'

icDir = dataDir + 'iowa-city-police/'
uiDir = dataDir + 'ui-police/'
gamesF = processDir + 'football/iowa-home-games.csv'

# output
icOut = processDir + 'iowa-city-police/'
uiOut = processDir + 'ui-police/'

In [4]:
# Dataframes
combDF = pd.read_pickle((processDir + 'combined-activity.pickle'))
homeGames = pd.read_csv(gamesF, parse_dates=['Date'], infer_datetime_format=True)

In [5]:
# functions

# pd.tslib.Timestamp('2012-09-29 00:00:00')
def lookupGame(date, gamesDF):
    # returns a single game from the games data frame
    return gamesDF[gamesDF['Date'] == date]

def lookupGameWeekend(date, gamesDF):
    day = date.dayofweek
    # Conditional to look up the saturday of a game weekend
    if (day >= 4):
        if (day == 6):
            adj = date - timedelta(days=1)
            return lookupGame(adj, gamesDF)
        elif (day == 4):
            adj = date + timedelta(days=1)
            return lookupGame(adj, gamesDF)
        else:
            return lookupGame(date, gamesDF)
        
def isGameWeekend(game):
    if (game is None) or (len(game) == 0):
        return
    if isinstance(game, pd.DataFrame):
        if game.size > 0:
            return True
        else:
            return False  
    
def isTeamWin(game):
    if (game is None) or (len(game) == 0):
        return
    if isinstance(game, pd.DataFrame):
        return game.iloc[0].Win.astype(int)

def lookupVisitTeam(game):
    if (game is None) or (len(game) == 0):
        return
    if isinstance(game, pd.DataFrame):
        return game.iloc[0]['Visitor']

In [6]:
combDF['gameWeekend'] = combDF.incDate.apply(lambda x: isGameWeekend(lookupGameWeekend(x, homeGames)))

In [7]:
combDF['gameWin'] = combDF.incDate.apply(lambda x: isTeamWin(lookupGameWeekend(x, homeGames)))

In [8]:
combDF['gameTeam'] = combDF.incDate.apply(lambda x: lookupVisitTeam(lookupGameWeekend(x, homeGames)))

In [9]:
combDF.head()

,name,age,incDate,timestamp,category,charge,incFormAddr,source,gameWeekend,gameWin,gameTeam
0,"Hamann, Brian Joel",25.0,2006-03-26,2006-03-26 03:54:46,PUBLIC INTOX,Public Intoxication,1 avenue south court street iowa city iowa,a,None,NaN,None
1,"PATHEUANGSIN, WALLY",16.0,2007-06-27,2007-06-27 08:47:05,OPERATING A VEHICLE,Operate Veh Without Owners Consent,1 avenue south court street iowa city iowa,a,None,NaN,None
2,"HOWARD, CURLEY LEE",56.0,2010-01-24,2010-01-24 01:05:39,DRIVING WHILE LIC SUSP,DRIVING WHILE BARRED,1 avenue south lower muscatine road iowa city...,a,None,NaN,None
3,"Montgomery, Christopher Joseph",23.0,2006-10-22,2006-10-22 15:01:01,POSSESSION:CONTROL SUBSTANCE,Possession Controlled Substance,1 avenue south lower muscatine road iowa city...,a,None,NaN,None
4,"Hildner, Johann Richard",23.0,2009-11-07,2009-11-07 16:21:17,POSSESSION:CONTROL SUBSTANCE,Possession Controlled Substance,1 avenue south lower muscatine road iowa city...,a,True,0.0,Northwestern


### Save out

In [10]:
combDF.to_pickle((processDir + 'combined-football.pickle'))
combDF.to_csv((processDir + 'combined-football.csv'), index=False)